In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
from tqdm import tqdm
import ssl
import sys
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import bs4.element
import time

In [2]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, page, maxpage_t, sort, s_date, e_date, s_from, e_to):
    urls = []
    if page == maxpage_t:       # 최대페이지 1일 경우
        url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        urls.append(url)
    else:
        while page <= maxpage_t :
            url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
            urls.append(url)
            page += 10
    return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(url, headers=headers, verify=False)

    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [3]:
def crawling_all(search, page, maxpage_t, sort, s_date, e_date, s_from, e_to) :
    # naver url 생성
    urls = makeUrl(search, page, maxpage_t, sort, s_date, e_date, s_from, e_to)

    #뉴스 크롤러 실행
    news_titles = []
    news_url =[]
    news_contents =[]
    news_dates = []

    while page <= maxpage_t :
        url_origin = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        url = articles_crawler(url_origin)
        news_url .append(url)
        page += 10

    #제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
    def makeList(newlist, content):
        for i in content:
            for j in i:
                newlist.append(j)
        return newlist

    #제목, 링크, 내용 담을 리스트 생성
    news_url_1 = []

    #1차원 리스트로 만들기(내용 제외)
    makeList(news_url_1,news_url)

    #NAVER 뉴스만 남기기
    final_urls = []
    for i in range(len(news_url_1)):
        if "news.naver.com" in news_url_1[i]:
            final_urls.append(news_url_1[i])
        else:
            pass
    
    for i in final_urls:
        #각 기사 html get하기
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
        news = requests.get(i, headers=headers, verify=False)
        news_html = BeautifulSoup(news.text,"html.parser")

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")
        
        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        # html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        # news_date = html_date.attrs['data-date-time']
        # news_dates.append(news_date)

        try:
            html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)

    a = pd.DataFrame({'date':news_dates,'title':news_titles,'content':news_contents})
    return a

In [4]:
import re

def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', d)
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\']'           
    text = re.sub(pattern, '', text)
    pattern = '[\"]'            
    text = re.sub(pattern, '', text)
    return text  

In [5]:
# keyword = ['비행체부품', '로켓', '우주기술', '국제시장', '기술 혁신', '연구 개발', '항공우주 정책과 규제', '전투기', '미사일']
# basename= ['past_news_aircraftparts', 'past_news_rocket', 'past_news_spacetech', 'past_news_internationalmarket', 'past_news_techrevol', 'past_news_randd', 'past_news_aerospacepolicyr', 'past_news_fighterplane', 'past_news_missile']

In [6]:
# keyword = ['전쟁', '무기수출', '미국']
# basename = ['past_news_war', 'past_news_armsexport', 'past_news_usa']

In [7]:
# day = []
# i = 1
# while i<7:
#     day.append('2023.{}.01'.format(str(i).zfill(2)))
#     i += 1

# print(day)

In [8]:
# day = ['2021.08.01', '2021.10.01', '2021.12.01']

In [9]:
# #데이터 프레임 만들기
# for i in tqdm(day) :
#     news_df = news_df = pd.DataFrame(
#         data=None
#         , index=None
#         , columns=['date','title','content']
#     )

#     # 최대 페이지 수
#     maxpage = 401
#     #검색어 입력
#     search = '사회'
#     # 검색 방식
#     sort = '0'
#     # 시작날짜
#     s_date = i
#     # 끝나는 날짜
#     e_1 = i[-2:]
#     e_2 = int(e_1)+30
#     e_3 = i[:-2]+str(e_2)
#     e_date = e_3

#     # 날짜 바꾸기
#     s_from = s_date.replace(".","")
#     e_to = e_date.replace(".","")

#     page = 1  
#     maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

#     a = crawling_all(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to)
#     news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link','content'])
#     news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

#     now = datetime.now()
#     news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\{}_date{}_{}.csv'.format(search, search,s_from, e_to),encoding='utf-8-sig',index=False)

In [10]:
news_df = news_df = pd.DataFrame(
    data=None
    , index=None
    , columns=['date','title','content']
)

# 최대 페이지 수
maxpage = 401
#검색어 입력
search = '경제'
# 검색 방식
sort = '0'
# 시작날짜
s_date = '2023.07.06'
# 끝나는 날짜
e_date = '2023.07.15'

# 날짜 바꾸기
s_from = s_date.replace(".","")
e_to = e_date.replace(".","")

page = 1  
maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

a = crawling_all(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to)
news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link','content'])
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

now = datetime.now()
news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\{}_date{}_{}.csv'.format(search, search,s_from, e_to),encoding='utf-8-sig',index=False)

In [11]:
# news_df = news_df = pd.DataFrame(
#     data=None
#     , index=None
#     , columns=['date','title','content']
# )

# # 최대 페이지 수
# maxpage = 401
# #검색어 입력
# search = '사회'
# # 검색 방식
# sort = '0'
# # 시작날짜
# s_date = '2021.11.01'
# # 끝나는 날짜
# e_date = '2021.11.31'

# # 날짜 바꾸기
# s_from = s_date.replace(".","")
# e_to = e_date.replace(".","")

# page = 1  
# maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

# a = crawling_all(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to)
# news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link','content'])
# news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

# now = datetime.now()
# news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\{}_date{}_{}.csv'.format(search, search,s_from, e_to),encoding='utf-8-sig',index=False)

In [12]:
folders = os.listdir('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}'.format(search))

In [13]:
df_all = pd.DataFrame()
for i in range(0,len(folders)):
    if folders[i].split('.')[1] == 'csv':
        file = 'D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\'.format(search)+folders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

In [14]:
df_all['content'] = df_all.content.apply(text_clean)
df_all['content'] = df_all.content.apply(clean_text)
df_all['title'] = df_all.content.apply(text_clean)
df_all['title'] = df_all.content.apply(clean_text)

df_all.dropna(axis=0, inplace=True)

df_all.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\0. merge\\{}_merge.csv'.format(search), index=False)

NewsDate = df_all['date'].to_list()
NewsTItle = df_all['title'].to_list()
NewsContent = df_all['content'].to_list()

import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

for date,title,content in zip(NewsDate, NewsTItle, NewsContent):
    sql = "INSERT INTO past_news_society VALUES ({}, {}, {})".format("\""+date+"\"", "\""+title+"\"", "\""+content+"\"")
    # print(sql)
    cur.execute(sql)
conn.commit()
conn.close()